In [0]:
# USER SEARCH TERM ENTERED IN : Cmd 52
# USER SEARCH QUERY ENTERED IN : Cmd 61

In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import math
from pyspark.sql.functions import *

In [0]:
import nltk
from nltk.corpus import stopwords


In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Out[3]: True

In [0]:
plot_summaries = sc.textFile("/FileStore/tables/plot_summaries.txt")

In [0]:
plot_summaries = plot_summaries.map(lambda x : x.lower())

In [0]:
#plot_summaries.collect()

In [0]:
#EXTRACTING DATA FROM MOVIE META FILE

In [0]:
movie_meta = sc.textFile("/FileStore/tables/movie_metadata.tsv")

In [0]:
#movie_meta.collect()

In [0]:
movie_meta=movie_meta.map(lambda x: x.lower())

In [0]:
m_meta=movie_meta.map(lambda x: (x.split('\t')[0],x.split('\t')[2]))

In [0]:
#m_meta.collect()

In [0]:
#plot_summaries.collect()

In [0]:
plot_summaries = plot_summaries.map(lambda x:x.split("\t"))

In [0]:
#plot_summaries.take(5)

In [0]:
#FINDING COUNT OF DOCUMENTS

In [0]:
movies_count = plot_summaries.count()

In [0]:
movies_count

Out[18]: 42306

In [0]:
###########################################CALCULATING TF-IDF################################################

In [0]:
psummaries=plot_summaries.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])


In [0]:
#psummaries.take(50)

In [0]:
#REMOVING STOP WORDS

In [0]:
stop_words = stopwords.words('english')

In [0]:
#CALCULATING TF

In [0]:
psummaries = psummaries.filter(lambda x: x[0][1] not in stop_words)

In [0]:
#test = psummaries.map(lambda x: x[0][1])
#test.take(5)

In [0]:
#psummaries.take(20)

In [0]:
reduced_psummaries=psummaries.reduceByKey(lambda x,y:x+y)

In [0]:
#reduced_psummaries.take(10)

In [0]:
term_freq = reduced_psummaries.map(lambda x: (x[0][1],(x[0][0],x[1])))

In [0]:
#term_freq.take(5)

In [0]:
mapped_psummaries = reduced_psummaries.map(lambda x: (x[0][1],(x[0][0],x[1],1)))

In [0]:
#mapped_psummaries.take(5)

In [0]:
mapped_plotsummaries=mapped_psummaries.map(lambda x:(x[0],x[1][2]))


In [0]:
#mapped_plotsummaries.take(5)

In [0]:
reduced_plotsummaries=mapped_plotsummaries.reduceByKey(lambda x,y:x+y)


In [0]:
#reduced_plotsummaries.take(5)

In [0]:
#CALCULATING IDF

In [0]:
inverse_doc_freq = reduced_plotsummaries.map(lambda x: (x[0],math.log10(movies_count/x[1])))


In [0]:
#inverse_doc_freq.take(50)

In [0]:
#JOINING TF, IDF TO CALCULATE TF-IDF

In [0]:
joined = term_freq.join(inverse_doc_freq)

In [0]:
#joined.take(50)

In [0]:
tf_idf=joined.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()


In [0]:
#tf_idf.take(5)

In [0]:
#JOINING TF-IDF WITH MOVIE META TO GET THE MOVIE NAMES WITH HIGHEST TF-IDF FOR SEARCH TERM

In [0]:
m_meta.collect()
tf_idf_movies = tf_idf.join(m_meta)
tf_idf_movies.collect()

Out[47]: [('10010424',
  (('submits', 1, 2.787552874323431, 2.787552874323431),
   'the bride of the regiment')),
 ('10010424',
  (('impersonate', 1, 2.7287748737702446, 2.7287748737702446),
   'the bride of the regiment')),
 ('10010424',
  (('disguise,', 1, 2.565704124707074, 2.565704124707074),
   'the bride of the regiment')),
 ('10010424',
  (('orders', 1, 1.2588596915401093, 1.2588596915401093),
   'the bride of the regiment')),
 ('10010424',
  (('champagne.', 1, 3.0700994642933987, 3.0700994642933987),
   'the bride of the regiment')),
 ('10010424',
  (('beltrami', 1, 4.626401965060686, 4.626401965060686),
   'the bride of the regiment')),
 ('10010424',
  (('steps', 1, 1.714711806306825, 1.714711806306825),
   'the bride of the regiment')),
 ('10010424',
  (('belief', 1, 2.1364434856358514, 2.1364434856358514),
   'the bride of the regiment')),
 ('10010424',
  (('led', 1, 1.2857595874999808, 1.2857595874999808),
   'the bride of the regiment')),
 ('10010424',
  (('husband,', 1, 1

In [0]:
tf_idf_movie = tf_idf_movies.map(lambda x: (x[0],(x[1][0][0],x[1][0][1],x[1][0][2],x[1][0][3],x[1][1])))

In [0]:
#tf_idf_movie.take(10)

In [0]:
#################### USER'S SEARCH TERM ##########################
search_term = "tamil"

In [0]:
result = tf_idf_movie.filter(lambda x : x[1][0]==search_term)

In [0]:
#result.take(10)


In [0]:
sorted_result = result.sortBy(lambda x: -x[1][3])

In [0]:
sorted_result.take(10)

Out[54]: [('32211779',
  ('tamil', 6, 2.4771828524053063, 14.863097114431838, 'flying fish')),
 ('2853320',
  ('tamil',
   5,
   2.4771828524053063,
   12.385914262026532,
   'kannathil muthamittal')),
 ('20533089', ('tamil', 4, 2.4771828524053063, 9.908731409621225, 'goa')),
 ('13865046', ('tamil', 3, 2.4771828524053063, 7.431548557215919, 'lanka')),
 ('14903088',
  ('tamil', 3, 2.4771828524053063, 7.431548557215919, 'naalu pennungal')),
 ('17489405', ('tamil', 3, 2.4771828524053063, 7.431548557215919, '1977')),
 ('10536562',
  ('tamil', 3, 2.4771828524053063, 7.431548557215919, 'rasathanthram')),
 ('27381948',
  ('tamil', 3, 2.4771828524053063, 7.431548557215919, 'ira madiyama')),
 ('28887962',
  ('tamil', 3, 2.4771828524053063, 7.431548557215919, 'chidambaram')),
 ('20604092',
  ('tamil', 2, 2.4771828524053063, 4.954365704810613, 'anbu thozhi'))]

In [0]:
top_10_movies = sorted_result.map(lambda x: x[1][4])

In [0]:
# TOP 10 MOVIES FOR USER'S SEARCH TERM

In [0]:
print("User search term : ",search_term)
print("\nTop 10 Recommendations: \n")
top_10_movies.take(10)

User search term :  tamil

Top 10 Recommendations: 

Out[57]: ['flying fish',
 'kannathil muthamittal',
 'goa',
 'lanka',
 'naalu pennungal',
 '1977',
 'rasathanthram',
 'ira madiyama',
 'chidambaram',
 'anbu thozhi']

In [0]:
######################################CALCULATING COSINE SIMILARITY##########################################

In [0]:
# USER'S SEARCH QUERY
user_query="Wall Street Mark romantic relationship with actress Anna due to Vermeer painting"

In [0]:
user_query=user_query.lower()

In [0]:
# READING-IN THE TF_IDF RDD AND NORMALIZING IT 

In [0]:
tf_idf2 = tf_idf

In [0]:
tf_idf2.take(5)

Out[63]: [('10000053', ('spends', 1, 1.5917735084353657, 1.5917735084353657)),
 ('10000053', ('broken,', 1, 2.6396302307944413, 2.6396302307944413)),
 ('10000053', ('nights', 1, 2.3934058546685324, 2.3934058546685324)),
 ('10000053', ('decides', 1, 0.7608834459859121, 0.7608834459859121)),
 ('10000053', ('women', 1, 1.3106414743949515, 1.3106414743949515))]

In [0]:
k_count=tf_idf2.countByKey()

In [0]:
kc_list = k_count.items()
    

In [0]:
kc_rdd = sc.parallelize(kc_list)

In [0]:
kc_rdd.take(10)

Out[67]: [('10000053', 275),
 ('10002175', 102),
 ('10002779', 84),
 ('10003264', 112),
 ('10004055', 98),
 ('10004330', 276),
 ('10004664', 181),
 ('10006475', 24),
 ('10007089', 47),
 ('10009844', 93)]

In [0]:
tf_idf3 = tf_idf2.join(kc_rdd)

In [0]:
#tf_idf3.take(15)

In [0]:
tf_idf4 = tf_idf3.map(lambda x:(x[0],(x[1][0][0],x[1][0][1]/x[1][1],x[1][0][2],x[1][0][3])))

In [0]:
#NORMALIZED TF-IDF
#tf_idf4.take(10)

In [0]:
tf_idf5 = tf_idf4.map(lambda x: (x[0],(x[1][0],x[1][1],x[1][2],x[1][1]*x[1][3])))

In [0]:
tf_idf5.take(5)

Out[73]: [('10016161',
  ('narrative',
   0.017241379310344827,
   2.3592302366576723,
   0.04067638339064952)),
 ('10016161',
  ('women', 0.017241379310344827, 1.3106414743949515, 0.022597266799912957)),
 ('10016161',
  ("hontvedt's",
   0.017241379310344827,
   4.626401965060686,
   0.07976555112173596)),
 ('10016161',
  ('photographer,',
   0.017241379310344827,
   2.605212665990748,
   0.044917459758461176)),
 ('10016161',
  ('immigrant', 0.034482758620689655, 2.3663305770756113, 0.1631952122121111))]

In [0]:
#CALCULATING NORMALIZED TF-IDF FOR USER'S QUERY

In [0]:
st = [(1,user_query)]

In [0]:
query_rdd = sc.parallelize(st)

In [0]:
query_rdd.collect()

Out[77]: [(1,
  'wall street mark romantic relationship with actress anna due to vermeer painting')]

In [0]:
qrdd = query_rdd.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])

In [0]:
qrdd = qrdd.filter(lambda x: x[0][1] not in stop_words)

In [0]:
qrdd.collect()

Out[80]: [((1, 'wall'), 1),
 ((1, 'street'), 1),
 ((1, 'mark'), 1),
 ((1, 'romantic'), 1),
 ((1, 'relationship'), 1),
 ((1, 'actress'), 1),
 ((1, 'anna'), 1),
 ((1, 'due'), 1),
 ((1, 'vermeer'), 1),
 ((1, 'painting'), 1)]

In [0]:
no_of_words = qrdd.count()

In [0]:
qrdd_reduced=qrdd.reduceByKey(lambda x,y:x+y)


In [0]:
qrdd_reduced.collect()

Out[83]: [((1, 'relationship'), 1),
 ((1, 'actress'), 1),
 ((1, 'anna'), 1),
 ((1, 'painting'), 1),
 ((1, 'vermeer'), 1),
 ((1, 'romantic'), 1),
 ((1, 'wall'), 1),
 ((1, 'street'), 1),
 ((1, 'due'), 1),
 ((1, 'mark'), 1)]

In [0]:
qrdd_mapped=qrdd_reduced.map(lambda x: (x[0][1],(x[0][0],x[1])))


In [0]:
qrdd_mapped.collect()

Out[85]: [('relationship', (1, 1)),
 ('actress', (1, 1)),
 ('anna', (1, 1)),
 ('painting', (1, 1)),
 ('vermeer', (1, 1)),
 ('romantic', (1, 1)),
 ('wall', (1, 1)),
 ('street', (1, 1)),
 ('due', (1, 1)),
 ('mark', (1, 1))]

In [0]:
qrdd_normalized_tf = qrdd_mapped.map(lambda x: (x[0],(x[1][0],x[1][1]/no_of_words)))

In [0]:
qrdd_normalized_tf.collect()

Out[87]: [('relationship', (1, 0.1)),
 ('actress', (1, 0.1)),
 ('anna', (1, 0.1)),
 ('painting', (1, 0.1)),
 ('vermeer', (1, 0.1)),
 ('romantic', (1, 0.1)),
 ('wall', (1, 0.1)),
 ('street', (1, 0.1)),
 ('due', (1, 0.1)),
 ('mark', (1, 0.1))]

In [0]:
qrdd_keys = qrdd_normalized_tf.keys().collect()
qrdd_keys

Out[88]: ['relationship',
 'actress',
 'anna',
 'painting',
 'vermeer',
 'romantic',
 'wall',
 'street',
 'due',
 'mark']

In [0]:
idf_value = tf_idf5.filter(lambda x: (x[1][0] in qrdd_keys))

In [0]:
idf_values = idf_value.map(lambda x: (x[1][0],x[1][2]))

In [0]:
#idf_values.collect()

In [0]:
idf_values_reduced = idf_values.reduceByKey(lambda x,y: x)

In [0]:
#idf_values_reduced.collect()

In [0]:
qrdd_tidf = qrdd_normalized_tf.join(idf_values_reduced)

In [0]:
#qrdd_tidf.collect()

In [0]:
qrdd_tf_idf = qrdd_tidf.map(lambda x: (x[0],(x[1][0][0],x[1][0][1],x[1][1],x[1][1]*x[1][0][1])))

In [0]:
documents_tf_idf = tf_idf5.map(lambda x: (x[1][0],(x[0],x[1][1],x[1][2],x[1][3])))

In [0]:
###########################USER QUERY TF-IDF (NORMALIZED)
qrdd_tf_idf.collect()

Out[98]: [('vermeer', (1, 0.1, 3.9274319607246673, 0.39274319607246677)),
 ('romantic', (1, 0.1, 1.5336813203765869, 0.1533681320376587)),
 ('mark', (1, 0.1, 1.8797677661231074, 0.18797677661231074)),
 ('relationship', (1, 0.1, 1.1038273323695094, 0.11038273323695094)),
 ('street', (1, 0.1, 1.4999458516288817, 0.1499945851628882)),
 ('wall', (1, 0.1, 1.7309794190212782, 0.17309794190212782)),
 ('anna', (1, 0.1, 2.125342702842935, 0.2125342702842935)),
 ('painting', (1, 0.1, 2.092375859004551, 0.20923758590045513)),
 ('actress', (1, 0.1, 1.8813271734786285, 0.18813271734786285)),
 ('due', (1, 0.1, 1.0913611518095256, 0.10913611518095256))]

In [0]:
#############################DOCUMENTS TF-IDF (NORMALIZED)
documents_tf_idf.collect()

Out[99]: [('narrative',
  ('10016161', 0.017241379310344827, 2.3592302366576723, 0.04067638339064952)),
 ('women',
  ('10016161',
   0.017241379310344827,
   1.3106414743949515,
   0.022597266799912957)),
 ("hontvedt's",
  ('10016161', 0.017241379310344827, 4.626401965060686, 0.07976555112173596)),
 ('photographer,',
  ('10016161', 0.017241379310344827, 2.605212665990748, 0.044917459758461176)),
 ('immigrant',
  ('10016161', 0.034482758620689655, 2.3663305770756113, 0.1631952122121111)),
 ('papers,',
  ('10016161', 0.017241379310344827, 2.8629739714977487, 0.04936162019823705)),
 ('two',
  ('10016161',
   0.017241379310344827,
   0.5170596954227128,
   0.008914822334874358)),
 ('1873.',
  ('10016161', 0.017241379310344827, 4.325371969396705, 0.0745753787827018)),
 ('plot',
  ('10016161',
   0.017241379310344827,
   1.3372591291283529,
   0.023056191881523327)),
 ('husband',
  ('10016161', 0.034482758620689655, 1.1442432696494098, 0.07891332894133861)),
 ('give',
  ('10016161',
   0.017

In [0]:
#################################################FINDING COSINE SIMILARITY#########################################

In [0]:
filtered_docs_tf_idf = documents_tf_idf.filter(lambda x: x[0] in qrdd_keys)

In [0]:
#filtered_docs_tf_idf.collect()

In [0]:
filtered_docs_tfidf = filtered_docs_tf_idf.map(lambda x: (x[0],(x[1][0],x[1][3])))

In [0]:
#filtered_docs_tfidf.collect()

In [0]:
qrdd_tfidf = qrdd_tf_idf.map(lambda x: (x[0],x[1][3]))

In [0]:
#qrdd_tfidf.collect()

In [0]:
query_documents_joined = filtered_docs_tfidf.join(qrdd_tfidf)

In [0]:
query_documents_joined.collect()

Out[108]: [('wall', (('10131333', 0.019839305662134994), 0.17309794190212782)),
 ('wall', (('10950125', 0.005769931396737594), 0.17309794190212782)),
 ('wall', (('11381533', 0.02884965698368797), 0.17309794190212782)),
 ('wall', (('1224344', 0.014922236370873088), 0.17309794190212782)),
 ('wall', (('1276915', 0.009356645508223125), 0.17309794190212782)),
 ('wall', (('14267167', 0.004862301738823815), 0.17309794190212782)),
 ('wall', (('14737096', 0.009891310965835875), 0.17309794190212782)),
 ('wall', (('15107077', 0.02012766766303812), 0.17309794190212782)),
 ('wall', (('161334', 0.005359069408734608), 0.17309794190212782)),
 ('wall', (('1721169', 0.005245392178852358), 0.17309794190212782)),
 ('wall', (('17529919', 0.006868965948497135), 0.17309794190212782)),
 ('wall', (('18717177', 0.005182573110842151), 0.17309794190212782)),
 ('wall', (('19344346', 0.004742409367181584), 0.17309794190212782)),
 ('wall', (('2019849', 0.007094177946808517), 0.17309794190212782)),
 ('wall', (('23322

In [0]:
query_documents_dot_prod = query_documents_joined.map(lambda x: (x[0],(x[1][0][0],x[1][0][1],x[1][1],x[1][1]*x[1][0][1])))

In [0]:
query_documents_dot_prod.collect()

Out[110]: [('wall',
  ('10131333',
   0.019839305662134994,
   0.17309794190212782,
   0.003434142978882799)),
 ('wall',
  ('10950125',
   0.005769931396737594,
   0.17309794190212782,
   0.0009987632496917473)),
 ('wall',
  ('11381533',
   0.02884965698368797,
   0.17309794190212782,
   0.004993816248458736)),
 ('wall',
  ('1224344',
   0.014922236370873088,
   0.17309794190212782,
   0.0025830084043752086)),
 ('wall',
  ('1276915',
   0.009356645508223125,
   0.17309794190212782,
   0.0016196160805812118)),
 ('wall',
  ('14267167',
   0.004862301738823815,
   0.17309794190212782,
   0.0008416544238975398)),
 ('wall',
  ('14737096',
   0.009891310965835875,
   0.17309794190212782,
   0.0017121655709001382)),
 ('wall',
  ('15107077',
   0.02012766766303812,
   0.17309794190212782,
   0.0034840578477619093)),
 ('wall',
  ('161334',
   0.005359069408734608,
   0.17309794190212782,
   0.0009276438851626137)),
 ('wall',
  ('1721169',
   0.005245392178852358,
   0.17309794190212782,
   0.00

In [0]:
query_documents_dt_squared = query_documents_dot_prod.map(lambda x: (x[1][0],(x[0],x[1][1]*x[1][1],x[1][3])))

In [0]:
#query_documents_dt_squared.collect()

In [0]:
query_documents_dt_squared_2 = query_documents_dt_squared.map(lambda x: (x[0],(x[1][1],x[1][2])))

In [0]:
query_documents_dt_squared_2.collect()

Out[114]: [('10131333', (0.0003935980491556216, 0.003434142978882799)),
 ('10950125', (3.3292108323058245e-05, 0.0009987632496917473)),
 ('11381533', (0.000832302708076456, 0.004993816248458736)),
 ('1224344', (0.0002226731383082076, 0.0025830084043752086)),
 ('1276915', (8.754681516655198e-05, 0.0016196160805812118)),
 ('14267167', (2.3641978199369094e-05, 0.0008416544238975398)),
 ('14737096', (9.783803262286503e-05, 0.0017121655709001382)),
 ('15107077', (0.0004051230055537104, 0.0034840578477619093)),
 ('161334', (2.87196249276351e-05, 0.0009276438851626137)),
 ('1721169', (2.7514139109965488e-05, 0.0009079665906288611)),
 ('17529919', (4.718269320161315e-05, 0.0011890038686806514)),
 ('18717177', (2.6859064049224086e-05, 0.0008970927392440845)),
 ('19344346', (2.2490446605931632e-05, 0.0008209013011165046)),
 ('2019849', (5.0327360740984306e-05, 0.0012279876020800172)),
 ('23322195', (3.490185964979489e-05, 0.0010226244877389904)),
 ('4114130', (0.00011277389999906814, 0.001838214

In [0]:
sqd = qrdd_tfidf.values().collect()

In [0]:
sqd

Out[116]: [0.39274319607246677,
 0.1533681320376587,
 0.18797677661231074,
 0.11038273323695094,
 0.1499945851628882,
 0.17309794190212782,
 0.2125342702842935,
 0.20923758590045513,
 0.18813271734786285,
 0.10913611518095256]

In [0]:
squared = [i ** 2 for i in sqd]
squared

Out[117]: [0.15424721806121608,
 0.023521783924720714,
 0.03533526854555458,
 0.012184347796859875,
 0.022498375578186916,
 0.02996289749075242,
 0.045170816045277125,
 0.04378036735345034,
 0.03539391933669086,
 0.011910691636790144]

In [0]:
sum_sqd=0
for j in squared:
    print(j)
    sum_sqd=sum_sqd+j
    
sum_sqd

0.15424721806121608
0.023521783924720714
0.03533526854555458
0.012184347796859875
0.022498375578186916
0.02996289749075242
0.045170816045277125
0.04378036735345034
0.03539391933669086
0.011910691636790144
Out[118]: 0.414005685769499

In [0]:
root = math.sqrt(sum_sqd)
root

Out[119]: 0.6434327360101435

In [0]:
query_documents_dt_squared_2=query_documents_dt_squared_2.sortByKey()
#query_documents_dt_squared_2.collect()

In [0]:
query_documents_dt_squared_2.count()

Out[121]: 11899

In [0]:
query_documents_reduced = query_documents_dt_squared_2.reduceByKey(lambda x,y : (x[0]+y[0],x[1]+y[1]))

In [0]:
query_documents_reduced = query_documents_reduced.sortByKey()
query_documents_reduced.collect()

Out[123]: [('10000053', (0.000257784548429432, 0.0017722687704523452)),
 ('10002175', (0.00011448184964235047, 0.0011677148663519748)),
 ('10004664', (0.0001994957932524748, 0.003610873857869794)),
 ('10010424', (0.0004750473888178204, 0.004560454932651076)),
 ('10014126', (0.00022864229305422917, 0.001669088739295873)),
 ('10015952', (1.284401648344986e-05, 0.0003955957076902557)),
 ('10020058', (8.940282707292285e-06, 0.00032632031881616835)),
 ('10026936', (2.9567200846562336e-05, 0.0006002141771852155)),
 ('100274', (0.0013407086256999696, 0.008748123782692302)),
 ('10034391', (1.3184240604289162e-05, 0.00040080091437039053)),
 ('10034593', (0.00019523069695337082, 0.0015423225059316295)),
 ('1004155', (0.0016316278166512595, 0.008810790209916802)),
 ('10044747', (0.0003336269781492661, 0.0019934211944418654)),
 ('10044772', (0.00033845410546832973, 0.0020307246328099787)),
 ('10047161', (3.663125029433275e-05, 0.0010476537584179167)),
 ('10048887', (8.393678602971785e-06, 0.000319

In [0]:
query_documents_reduced.count()

Out[124]: 9887

In [0]:
query_documents_reduced2 = query_documents_reduced.map(lambda x: (x[0],(math.sqrt(x[1][0]),x[1][1])))

In [0]:
query_documents_reduced3 = query_documents_reduced2.map(lambda x:(x[0],(x[1][0]*root,x[1][1])))

In [0]:
#query_documents_reduced3.collect()

In [0]:
#query_documents_final = query_documents_reduced3.map(lambda x: (x[0],x[1][0]*x[1][1]))

In [0]:
query_documents_final = query_documents_reduced3.map(lambda x: (x[0],x[1][1]/x[1][0]))

In [0]:
#query_documents_final.collect()

In [0]:
op = query_documents_final.sortBy(lambda x:-x[1])

In [0]:
#op.collect()

In [0]:
output=op.join(m_meta)

In [0]:
#output.collect()

In [0]:
output = output.sortBy(lambda x:-x[1][0])

In [0]:
output.collect()

Out[136]: [('32583135', (0.8209580308315574, 'all the vermeers in new york')),
 ('1630107', (0.6340371378529062, 'a zed & two noughts')),
 ('5959715', (0.6252612977722062, 'girl with a pearl earring')),
 ('12635192', (0.6103873397984452, 'amsterdamned')),
 ('13281115', (0.5134323387949474, 'how about you')),
 ('686033', (0.5111104864230196, 'cradle will rock')),
 ('5454803', (0.5048261509196215, 'the lives of others')),
 ('5099214', (0.4955630817304638, 'the day of the locust')),
 ('396282', (0.4954201260972183, 'beat street')),
 ('1006385', (0.46960444302074167, 'sirens')),
 ('2045991', (0.4690511735378381, 'now and then')),
 ('17752771', (0.46573222587891366, 'beyond the clouds')),
 ('161901', (0.463524519472028, 'charly')),
 ('15107077', (0.4614778663250861, 'the helen morgan story')),
 ('3117881', (0.46050627426510465, 'la mariée était en noir')),
 ('26499456', (0.4592497088051348, 'bedrooms')),
 ('2051753', (0.4477792833801887, 'stuck on you')),
 ('44302', (0.4428833424822666, 'am

In [0]:
top_10_movies_for_query = output.map(lambda x: x[1][1])

In [0]:
print("User's Search Query : ",user_query)
print("Top 10 Movie Recommendations are : ")
top_10_movies_for_query.take(10)

User's Search Query :  wall street mark romantic relationship with actress anna due to vermeer painting
Top 10 Movie Recommendations are : 
Out[138]: ['all the vermeers in new york',
 'a zed & two noughts',
 'girl with a pearl earring',
 'amsterdamned',
 'how about you',
 'cradle will rock',
 'the lives of others',
 'the day of the locust',
 'beat street',
 'sirens']